In [1]:
def pxlocal(line, cell):
    ip = get_ipython()
    ip.run_cell_magic("px", line, cell)
    ip.run_cell(cell)
get_ipython().register_magic_function(pxlocal, "cell")

In [2]:
import subprocess
import ipyparallel as ipp
import time
from IPython.display import clear_output

subprocess.Popen(["ipcluster", "stop"])

num_engines = 8
subprocess.Popen(["ipcluster", "start", "-n={:d}".format(num_engines)])

wait_time = 30
# Waiting in order for clusters to start properly
for i in range(wait_time):
    clear_output(wait = True)
    print('Waiting for', wait_time - (i+1), 'seconds')
    time.sleep(1)
    
c = ipp.Client()

Waiting for 1 seconds


In [3]:
%%pxlocal

from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from gym import wrappers
from IPython.display import Video
import io
import base64
from IPython.display import HTML
import numpy as np

class Agent:
    
    def __init__(self):
        self.state = None
        self.reward = None
        self.done = None
        self.info = None
    
    def make_env(self, mode = None):
        env = gym_super_mario_bros.make('SuperMarioBros-v0')
        env = JoypadSpace(env, SIMPLE_MOVEMENT)
        if mode == 'monitor':
            env = wrappers.Monitor(env, directory, force = True)
        env.reset()
        return env
    
    def reset_data(self):
        self.state = []
        self.reward = []
        self.done = []
        self.info = []
    
    def update(self, state, reward, done, info):
        self.state.append(state)
        self.reward.append(reward)
        self.done.append(done)
        self.done.append(info)
            
        
    def run(self, max_steps = 500, mode = None, directory = './gym-results/'):    
        env = self.make_env(mode = mode)
        self.reset_data()
        
        for step in range(max_steps):
            state, reward, done, info = env.step(env.action_space.sample())
            self.update(state, reward, done, info)
            
            if mode == 'render':
                env.render()

        if mode == 'monitor':
            file_name = directory + 'openaigym.video.%s.video000000.mp4'% env.file_infix
            mp4 = Video(file_name, width = 600, height = 450)
            display(mp4)

        if mode == 'render':    
            env.close()
    
    def get_reward(self):
        if self.reward == None:
            self.run()
        return self.reward
    
    
    def itsame(self):
        return 'Mario!'
    
    
mario = Agent()

In [4]:
par_dict = dict()

c[:].push(par_dict)

<AsyncResult: _push>

In [5]:
start_time = time.time()

ans = c[:].apply_async(lambda i: sum(mario.get_reward()), c.ids).get()

end_time = time.time()

print('Parallel time:', round(end_time - start_time, 3))
print('Ans:', ans)

Parallel time: 3.575
Ans: [866, 684, 528, 529, 529, 808, 529, 528]


In [6]:
start_time = time.time()

ans = [sum(Agent().get_reward()) for i in c.ids]

end_time = time.time()

print('Parallel time:', round(end_time - start_time, 3))
print('Ans:', ans)

D:\Anaconda3\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


Parallel time: 10.554
Ans: [528, 529, 529, 623, 529, 369, 529, 767]


In [7]:
subprocess.Popen(["ipcluster", "stop"])